# Setup Prefect working env

To work with Prefect, you need three things:
- Prefect server: Coordinates with worker to run registered workflow
- Prefect worker: The compute module which runs the registered workflow
- Prefect workflow: The tasks logics which are defined by users.

The `Prefect server` is managed by CASD inside each bulle, run as daemon service.

Users need to manage their workers and workflows.

## 1. Prepare your python virtual env for prefect

```powershell
# open a conda shell with python 11

# create a dedicated virtual env if you don't have one
conda create --name <username>-prefect python=python-version --offline

# for example
# change pengfei to your name
conda create --name pengfei-prefect python --offline

# activate your virtual env
conda activate pengfei-prefect

# install prefect package
pip install prefect

# check the installed prefect version
prefect version

# expected output
Version:              3.4.25
API version:          0.8.4
Python version:       3.11.8
...
```

> This virtual env will be used by your prefect worker, if your workflow requires a python package, you need to install the package inside this virtual env. For example, if my workflow uses pyspark, I need to install pyspark inside this virtual env
## 2. Register a prefect worker in the prefect server

Why CASD does not propose a general worker for everyone? As a workflow may call any python package, we can pre-install all required python packages in a worker, which will slow down the worker. So we propose users to create their own worker and manager their own python package in their worker.

### 2.1 Check prefect server availability

Normally, the prefect server runs on `localhost:4200`. You can try to access the Prefect server in your browser by typing `http://localhost:4200`.

### 2.2 Configure your registry for the prefect server

Open a new powershell terminal, and type the below commands

```powershell
setx PREFECT_HOME "C:\Users\Public\Documents\prefect_home"
setx PREFECT_API_URL "http://localhost:4200/api"
```

### 2.3. Create and register a new worker pool

Go back to your virtual env terminal

```powershell
# create a new work pool with your user name
prefect work-pool create "%USERNAME%-pool" --type process

# the you can activate your worker
prefect worker start --pool "%USERNAME%-pool"

# expected output
Discovered type 'process' for work pool 'pliu-pool'.
Worker 'ProcessWorker a8986b12-18f9-40d6-863f-5e08681e63b4' started!
```
> The worker runs as process, so if you close your terminal, the process will be killed, and your worker will be stopped.

> Go back to your browser, and open `http://localhost:4200`. You should see the below figure

![prefect_work_pool.png](../assets/prefect_work_pool.png)

## 3. Create and run a simple workflow to test your worker

We recommend you to save your workflows in a directory which can be shared with other users, yet you can set some workflow private if you want.
Open a new powershell, and type the below  command

``` powershell
# create a personal dir to store your workflow
New-Item -Path "C:\Users\Public\Documents\prefect_home\flows\$Env:UserName" -Type Directory

# for example, if my username is pliu, a directory pliu will be created under flows. The full path will be
cd C:\Users\Public\Documents\prefect_home\flows\pliu
```

Create a file called `simple_etl.py` under `C:\Users\Public\Documents\prefect_home\flows\pliu`, and put the below command in it.

```python
from typing import List
from prefect import flow, task

@task
def extract()->List[int]:
    data = [1, 2, 3]
    msg = f"Task1: Extracting data: {data}"
    print(msg)
    return data

@task
def transform(data:List[int])->List[int]:
    print(f"Task2: Transforming data: {data}")
    return [x * 10 for x in data]

@task
def load(data:List[int]):
    print(f"Task3: Loading result {data}")

@flow
def etl_flow():
    # task 1
    data = extract()
    # task 2
    clean = transform(data)
    # task 3
    load(clean)


if __name__ == "__main__":
    etl_flow()

```

### 3.1 Run your workflow

Go back to your virtual env terminal, and run your workflow as a single run (This mode is for test purpose only.)


```powershell
# replace pliu by your own user name
cd C:\Users\Public\Documents\prefect_home\flows\pliu

# start your workflow as a single run
python simple_etl.py
```

> Go back to your browser, and open `http://localhost:4200`. Click on tab `Runs`, You should see the below figure

![simple_run.png](../assets/simple_run.png)


> The run name started with the flow name and followed by auto-generated string. Click on it, you will see the output of your workflow

### 3.2 Register your workflow as a deployment

